# Fine align
---

#### Overview
Interactive 3D fine alignment of serial sections. Iterate through the z-levels, sample a cone with a radius of 0.1 times the image size to find overlapping tiles in z between neighboring z levels. Point matches are then sought in z for every tile pair. The alignment is solved on the full set of intra-ROA and inter-ROA point matches, for a similarity transformation model, with weights given to the intra-ROA and inter-ROA matches, respectively.

In [11]:
import pathlib
import requests

from tqdm.notebook import tqdm
import numpy as np
import renderapi

from scripted_render_pipeline import basic_auth
from interactive_render.utils import clear_image_cache
from interactive_render.utils import get_stitching_pointmatches, get_alignment_pointmatches

clear_image_cache()

<Response [200]>

#### parameters and `render-ws` environment variables
* `host` : Localhost or Web address which hosts render-ws (for uploader). It's usually the preamble of the link to render-ws html page, i.e. `{host_name}/render-ws/view/index.html`
* `owner` : render-ws ID of dataset
* `project`: render-ws project name
* `auth`: authorization for render-ws. See https://github.com/hoogenboom-group/scripted-render-pipeline/tree/master for instructions
* `dir_project`: full path to project directory on disk
* `memGB`: Gigabytes of RAM to give to java PointMatchClient. Increase proportionally if using more threads to find matches

In [12]:
# Set script parameters 
num_threads = 20
owner = "akievits"
project = "20231107_MCF7_UAC_test"

# set project directory
dir_project = pathlib.Path("/long_term_storage/akievits/FAST-EM/tests/20231107_MCF7_UAC_test/")

# create an authorized session
auth = basic_auth.load_auth()
sesh = requests.Session()
sesh.auth = auth

# render-ws environment variables
params_render = {
    "host": "http://localhost",
    "port": 8081,
    "client_scripts": "/home/catmaid/render/render-ws-java-client/src/main/scripts",
    "client_script": "/home/catmaid/render/render-ws-java-client/src/main/scripts/run_ws_client.sh",
    "owner": owner,
    "project": project,
    "session": sesh
}

params_align = {
    "host": "http://localhost",
    "port": 8081,
    "client_scripts": "/home/catmaid/render/render-ws-java-client/src/main/scripts",
    "client_script": "/home/catmaid/render/render-ws-java-client/src/main/scripts/run_ws_client.sh",
    "owner": owner,
    "project": project,
    "memGB": f"{num_threads * 2}G",
    "session": sesh
}

params_uploader = {
    "host": "https://sonic.tnw.tudelft.nl",
    "owner": owner,
    "project": project,
    "auth": auth
}

## 5) Fine alignment
---

In [13]:
# Stack from rough alignment, stack to align ('in') and output stack ('out')
stacks = {
    'rough_align': 'postcorrection_aligned_rough',
    'in': 'postcorrection',
    'out': 'postcorrection_rigid'
}

### Get point matches

Use `render-ws` `PointMatchClient` script to find point matches between tilepairs in `z`, based on rough alignment
#### Collect tile pairs

In [6]:
# Get z values
z_values = renderapi.stack.get_z_values_for_stack(stack=stacks['rough_align'],
                                                  **params_render)

# Get tile pairs
tile_pairs = renderapi.client.tilePairClient(stack=stacks['rough_align'],
                                             minz=min(z_values),
                                             maxz=max(z_values),
                                             zNeighborDistance=1, # half-height of search cylinder
                                             xyNeighborFactor=0.1,
                                             excludeSameLayerNeighbors=True,
                                             subprocess_mode="check_output",  # suppresses output
                                             **params_render)
# Reformat tilepairs for PointMatchClient
tile_pairs_reformat = [[tp['p']['id'], tp['q']['id']] for tp in tile_pairs['neighborPairs']]

# Show tile pairs
out = f"Number of tile pairs... {len(tile_pairs_reformat)}"
print(out, "\n" + "-"*len(out))
tile_pairs_reformat[0:5]

Number of tile pairs... 64 
--------------------------


[['t00_z0_y4_x4', 't00_z1_y4_x4'],
 ['t00_z0_y4_x4', 't01_z1_y4_x3'],
 ['t01_z0_y4_x3', 't01_z1_y4_x3'],
 ['t01_z0_y4_x3', 't02_z1_y4_x2'],
 ['t02_z0_y4_x2', 't02_z1_y4_x2']]

In [14]:
# Name for pointmatch collection
match_collection = f"{params_render['project']}_{stacks['in']}_align_matches"
match_collection

'20231107_MCF7_UAC_test_postcorrection_align_matches'

#### Set `SIFT` & `RANSAC` parameters for fine alignment

In [8]:
from renderapi.client import (
    SiftPointMatchOptions,
    MatchDerivationParameters,
    FeatureExtractionParameters
)

In [9]:
# `RANSAC` parameters
params_RANSAC = MatchDerivationParameters(
    matchIterations=None,
    matchMaxEpsilon=25,        # maximal alignment error
    matchMaxNumInliers=None,
    matchMaxTrust=None,
    matchMinInlierRatio=0.05,  # minimal inlier ratio
    matchMinNumInliers=7,      # minimal number of inliers
    matchModelType='AFFINE',   # expected transformation
    matchRod=0.92              # closest/next closest ratio
)

# `SIFT` parameters
params_SIFT = FeatureExtractionParameters(
    SIFTfdSize=8,              # feature descriptor size
    SIFTmaxScale=0.2,         # (width/height *) maximum image size
    SIFTminScale=0.05,         # (width/height *) minimum image size
    SIFTsteps=5               # steps per scale octave
)

# Combined `SIFT` & `RANSAC` parameters
params_SIFT = SiftPointMatchOptions(
    fillWithNoise=True,
    **{**params_RANSAC.__dict__,
       **params_SIFT.__dict__}
)

# Extra parameters
params_SIFT.numberOfThreads = num_threads # multithreading
params_SIFT.__dict__

{'SIFTfdSize': 8,
 'SIFTmaxScale': 0.2,
 'SIFTminScale': 0.05,
 'SIFTsteps': 5,
 'matchIterations': None,
 'matchMaxEpsilon': 25,
 'matchMaxNumInliers': None,
 'matchMaxTrust': None,
 'matchMinInlierRatio': 0.05,
 'matchMinNumInliers': 7,
 'matchModelType': 'AFFINE',
 'matchRod': 0.92,
 'renderScale': None,
 'fillWithNoise': True,
 'numberOfThreads': 20}

#### Get point matches

In [10]:
batch_size = params_SIFT.numberOfThreads # batch size equal to # of threads used
batches = [i for i in range(0, len(tile_pairs_reformat)+1, batch_size)]

In [12]:
for i in tqdm((batches),
              total=len(batches)):
    tile_pair_batch = tile_pairs_reformat[i:(i+batch_size)]

    # Run SIFT + RANSAC via render-ws PointMatchClient
    renderapi.client.pointMatchClient(
        stack=stacks['in'],
        collection=match_collection,
        tile_pairs=tile_pair_batch,
        sift_options=params_SIFT,
        excludeAllTransforms=True,
        subprocess_mode='check_output',  # suppresses output
        **params_align
    )

  0%|          | 0/4 [00:00<?, ?it/s]

In [13]:
# renderapi.pointmatch.delete_collection(match_collection,
#                                        **params_render)

#### Inspect matches

In [14]:
from interactive_render import plotting

In [15]:
plotting.plot_aligned_stack_with_alignment_matches(
    stacks['in'],
    match_collection,
    width=1000,
    **params_render
)

interactive(children=(IntSlider(value=0, description='z', max=1), Output()), _dom_classes=('widget-interact',)…

### Combine stitching and aligment match collections
Facilitate alignment using both match collections

In [15]:
# Name for stitch and align pointmatch collection
match_collection_total = f"{params_render['project']}_{stacks['in']}_total_matches"
match_collection_total

'20231107_MCF7_UAC_test_postcorrection_total_matches'

In [22]:
# Get all point matches
# get intrasection point matches
stitch_matches = get_stitching_pointmatches(
    "postcorrection",
    f"{params_render['project']}_{stacks['in']}_stitch_matches",
    **params_render
    )

# get intersection point matches
align_matches = get_alignment_pointmatches(
    "postcorrection",
    f"{params_render['project']}_{stacks['in']}_align_matches",
    **params_render
    )

# Combine the lot
all_matches = []
z_values = renderapi.stack.get_z_values_for_stack(stack="postcorrection",
                                                  **params_render)
for z in z_values:
    all_matches += stitch_matches[z]

for z in z_values[:-1]:
    all_matches += align_matches[z]
    
# Import total collection of matches
renderapi.pointmatch.import_matches(
    match_collection_total,
    all_matches,
    **params_render
)

<Response [201]>

### Align stack
---

### Create alignment files
The alignment file is generated on disk and contains the settings for this alignment job. The regularization parameters are the most important parameters that may need adjustment, if alignment fails a good strategy is to decrease or increase the regularization parameters by a factor of 10 each time. Larger values increase regulariation, smaller values decrease it and lead to more flexible outcomes

In [16]:
from pathlib import Path
import os
import subprocess
import json
from pprint import pprint

In [17]:
z_values = renderapi.stack.get_z_values_for_stack(stacks['in'],
                                                  **params_render)
# Load align.json template
template_align_json = Path(f"/home/{owner}/interactive-render-workflow/templates/align.json") # Replace with path to installation
with template_align_json.open('r') as json_data:
    params_align_fine = json.load(json_data)

# Edit BigFeta solver schema
params_align_fine['first_section'] = min(z_values)
params_align_fine['last_section'] = max(z_values)
params_align_fine['solve_type'] = '3D'
params_align_fine['transformation'] = 'SimilarityModel'
params_align_fine['log_level'] = 'INFO'

# Edit input stack data
params_align_fine['input_stack']['host'] = params_render['host']
params_align_fine['input_stack']['owner'] = params_render['owner']
params_align_fine['input_stack']['project'] = params_render['project']
params_align_fine['input_stack']['name'] = stacks['in']

# Edit point match stack data
params_align_fine['pointmatch']['host'] = params_render['host']
params_align_fine['pointmatch']['owner'] = params_render['owner']
params_align_fine['pointmatch']['name'] = match_collection_total

# Edit output stack data
params_align_fine['output_stack']['host'] = params_render['host']
params_align_fine['output_stack']['owner'] = params_render['owner']
params_align_fine['output_stack']['project'] = params_render['project']
params_align_fine['output_stack']['name'] = stacks['out']

# Edit alignment parameters
params_align_fine['matrix_assembly']['depth'] = 2
params_align_fine['matrix_assembly']['montage_pt_weight'] = 0.5
params_align_fine['matrix_assembly']['cross_pt_weight'] = 1
params_align_fine['matrix_assembly']['npts_min'] = 5
params_align_fine['matrix_assembly']['npts_max'] = 100

# Edit regularization parameters
params_align_fine['regularization']['default_lambda'] = 1e3    # default: 0.005
params_align_fine['regularization']['translation_factor'] = 0.00001  # default: 0.005
params_align_fine['regularization']['thinplate_factor'] = 1e-5      # default: 1e-5

# Export montage settings to
align_json = dir_project / '_jsons_align_fine' / stacks['in'] / 'align_fine.json'
align_json.parent.mkdir(parents=True, exist_ok=True)
with align_json.open('w') as json_data:
    json.dump(params_align_fine, json_data, indent=2)

# Check alignment parameters
print(align_json)
print('-'*len(align_json.as_posix()))
pprint(params_align_fine)

/long_term_storage/akievits/FAST-EM/tests/20231107_MCF7_UAC_test/_jsons_align_fine/postcorrection/align_fine.json
-----------------------------------------------------------------------------------------------------------------
{'close_stack': 'True',
 'first_section': 0.0,
 'hdf5_options': {'chunks_per_file': -1, 'output_dir': ''},
 'input_stack': {'client_scripts': '/home/catmaid/render/render-ws-java-client/src/main/scripts',
                 'collection_type': 'stack',
                 'db_interface': 'render',
                 'host': 'http://localhost',
                 'mongo_host': 'sonic.tnw.tudelft.nl',
                 'mongo_port': 27017,
                 'name': 'postcorrection',
                 'owner': 'akievits',
                 'port': 8081,
                 'project': '20231107_MCF7_UAC_test'},
 'last_section': 2.0,
 'log_level': 'INFO',
 'matrix_assembly': {'choose_random': 'False',
                     'cross_pt_weight': 1,
                     'depth': 2,
       

### Run `BigFeta`

In [20]:
# Path to `BigFeta`
cwd = Path.cwd().as_posix()
BigFeta_dir = Path(f"/home/{owner}/BigFeta/")

# Call `BigFeta.BigFeta` process -- have to switch to BigFeta directory
os.chdir(BigFeta_dir.as_posix())
subprocess.run(['python', '-m', 'bigfeta.bigfeta', '--input_json', align_json.as_posix()])
os.chdir(cwd)

/home/akievits/interactive-render-workflow/.venv/lib/python3.10/site-packages/argschema/utils.py:346: FutureWarning: '--transform_apply' is using old-style command-line syntax with each element as a separate argument. This will not be supported in argschema after 2.0. See http://argschema.readthedocs.io/en/master/user/intro.html#command-line-specification for details.
  warnings.warn(warn_msg, FutureWarning)
INFO:bigfeta.utils:
 loaded 75 tile specs from 3 zvalues in 0.0 sec using interface: render
INFO:__main__: A created in 0.3 seconds
INFO:__main__:
 solved in 0.0 sec
 precision [norm(Kx-Lm)/norm(Lm)] = 3.6e-09
 error     [norm(Ax-b)] = 1360.075
 [mean(Ax) +/- std(Ax)] : 0.1 +/- 6.3
 [mean(error mag) +/- std(error mag)] : 4.3 +/- 4.6
/home/catmaid/BigFeta/bigfeta/transform/similarity_model.py:68: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this oper

In [26]:
clear_image_cache()

<Response [200]>

### Inspect fine alignment

In [27]:
# plot stack
plotting.plot_stacks(
    stacks=[stacks['out']],
    width=1000,
    vmin=0,
    vmax=65535,
    **params_render
)

  0%|          | 0/1 [00:00<?, ?it/s]

interactive(children=(IntSlider(value=0, description='z', max=2), IntSlider(value=28550, description='vmin', m…

### Inspect fine alignment
Plot tiles to qualitatively assess alignment quality

In [28]:
plotting.plot_aligned_tiles(stack=stacks['out'],
                            width=2000,
                            **params_render)

interactive(children=(IntSlider(value=0, description='z', max=1), FloatSlider(value=0.5, description='alpha1',…

### Update stack scale and bounds
Set minimum X and Y bounds to 0 because WebKnossos doesn't allow for negative coordinates.  
Then scale `tforms` such that tile sizes are back approximately to the original size before alignment

In [35]:
bounds = renderapi.stack.get_stack_bounds(f"{stacks['out']}",
                                          **params_render)
tX = -1*bounds['minX']
tY = -1*bounds['minY']

ts_new = []
z_values = renderapi.stack.get_z_values_for_stack(f"{stacks['out']}",
                                                  **params_render)
for z in tqdm(z_values, total=len(z_values)):
    # Get tilespec and transformations from the aligned stack
    TileSpec = renderapi.tilespec.get_tile_specs_from_z(f"{stacks['out']}",
                                                        z=z,
                                                        **params_render)
    translation = renderapi.transform.leaf.AffineModel(B0=tX, B1=tY)
    # Translate stack
    for ts in TileSpec:
        ts.tforms += [translation]
        ts_new.append(ts)
        
# Create new stack
renderapi.stack.create_stack(f"{stacks['out']}_translated",
                             **params_render)
# Import tilespecs to stack
renderapi.client.import_tilespecs(f"{stacks['out']}_translated",
                                  tilespecs=ts_new,
                                  **params_render)
# Close stack
renderapi.stack.set_stack_state(f"{stacks['out']}_translated",
                                state='COMPLETE',
                                **params_render)

  0%|          | 0/3 [00:00<?, ?it/s]

org.janelia.render.client.ImportJsonClient

  Running: /home/catmaid/render/deploy/jdk1.8.0_131/bin/java -cp /home/catmaid/render/render-ws-java-client/target/render-ws-java-client-2.0.1-SNAPSHOT-standalone.jar -Xms1G -Xmx1G -Djava.awt.headless=true -XX:+UseSerialGC org.janelia.render.client.ImportJsonClient --baseDataUrl http://localhost:8081/render-ws/v1 --owner akievits --project 20231107_MCF7_UAC_test --stack postcorrection_rigid_translated /tmp/tmpxvdoqkt8.json


16:39:26.348 [main] INFO  [org.janelia.render.client.ClientRunner] run: entry
16:39:26.515 [main] INFO  [org.janelia.render.client.ImportJsonClient] runClient: entry, parameters={
  "renderWeb" : {
    "baseDataUrl" : "http://localhost:8081/render-ws/v1",
    "owner" : "akievits",
    "project" : "20231107_MCF7_UAC_test"
  },
  "tileSpecValidator" : { },
  "stack" : "postcorrection_rigid_translated",
  "tileFiles" : [
    "/tmp/tmpxvdoqkt8.json"
  ]
}
16:39:26.816 [main] INFO  [org.janelia.render.client.RenderDataClient] 

<Response [201]>

Determine scaling factor

In [36]:
from matplotlib.transforms import Affine2D as AffineMPL
from renderapi.transform import AffineModel as AffineRender

In [37]:
# Collect scaling factors
scales_x = []
scales_y = []

z_values = renderapi.stack.get_z_values_for_stack(stacks['out'],
                                                  **params_render)
for z in tqdm(z_values, total=len(z_values)):
    # Get tilespec and transformations from the aligned stack
    TileSpec = renderapi.tilespec.get_tile_specs_from_z(stacks['out'],
                                                        z=z,
                                                        **params_render)
    # tform scale terms (have to isolate them)
    M00 = np.array([ts.tforms[0].M[0, 0] for ts in TileSpec])
    M01 = np.array([ts.tforms[0].M[0, 1] for ts in TileSpec])
    M10 = np.array([ts.tforms[0].M[1, 0] for ts in TileSpec])
    M11 = np.array([ts.tforms[0].M[1, 1] for ts in TileSpec])
    
    # Extract scale from affine transform
    M = AffineMPL(np.array([[M00.mean(), M01.mean(), 0],
                            [M10.mean(), M11.mean(), 0],
                            [         0,          0, 1]]))
    R, S = np.linalg.qr(M.get_matrix())
    mask = np.diag(S) < 0.
    S[mask, :] *= -1.
    # Set scale
    sx = 1/S[0,0]
    sy = 1/S[1,1]
    
    # Collect scaling factors
    scales_x.append(sx)
    scales_y.append(sy)
    
# Put into arrays
scales_x = np.array(scales_x)
scales_y = np.array(scales_y)
# Find average
sx = scales_x.mean()
sy = scales_y.mean()

# Out
sx, sy

  0%|          | 0/3 [00:00<?, ?it/s]

(0.9640213140573337, 0.9640213140573337)

Scale tileset by scaling factor

In [38]:
ts_new = []
z_values = renderapi.stack.get_z_values_for_stack(stacks['out'],
                                                  **params_render)
for z in tqdm(z_values, total=len(z_values)):
    # Get tilespec and transformations from the aligned stack
    TileSpec = renderapi.tilespec.get_tile_specs_from_z(f"{stacks['out']}_translated",
                                                        z=z,
                                                        **params_render)
    # Scale 
    S = renderapi.transform.leaf.AffineModel(M00=sx,
                                             M11=sy)
    # Add to tforms
    for ts in TileSpec:
        ts.tforms += [S]
        ts_new.append(ts)
        
# Create new stack
renderapi.stack.create_stack(f"{stacks['out']}_scaled",
                             **params_render)
# Import tilespecs to stack
renderapi.client.import_tilespecs(f"{stacks['out']}_scaled",
                                  tilespecs=ts_new,
                                  **params_render)
# Close stack
renderapi.stack.set_stack_state(f"{stacks['out']}_scaled",
                                state='COMPLETE',
                                **params_render)

  0%|          | 0/3 [00:00<?, ?it/s]

org.janelia.render.client.ImportJsonClient

  Running: /home/catmaid/render/deploy/jdk1.8.0_131/bin/java -cp /home/catmaid/render/render-ws-java-client/target/render-ws-java-client-2.0.1-SNAPSHOT-standalone.jar -Xms1G -Xmx1G -Djava.awt.headless=true -XX:+UseSerialGC org.janelia.render.client.ImportJsonClient --baseDataUrl http://localhost:8081/render-ws/v1 --owner akievits --project 20231107_MCF7_UAC_test --stack postcorrection_rigid_scaled /tmp/tmp6wuea3zg.json


16:39:28.399 [main] INFO  [org.janelia.render.client.ClientRunner] run: entry
16:39:28.560 [main] INFO  [org.janelia.render.client.ImportJsonClient] runClient: entry, parameters={
  "renderWeb" : {
    "baseDataUrl" : "http://localhost:8081/render-ws/v1",
    "owner" : "akievits",
    "project" : "20231107_MCF7_UAC_test"
  },
  "tileSpecValidator" : { },
  "stack" : "postcorrection_rigid_scaled",
  "tileFiles" : [
    "/tmp/tmp6wuea3zg.json"
  ]
}
16:39:28.857 [main] INFO  [org.janelia.render.client.RenderDataClient] getStack

<Response [201]>

### Update stack metadata 
Set final pixel resolution of stack

In [39]:
from interactive_render.utils import update_stack_resolution

In [42]:
update_stack_resolution(stack=f"{stacks['out']}_scaled",
                        stackresolutionX=4.0,
                        stackresolutionY=4.0,
                        stackresolutionZ=100.0,
                        **params_render)

In [44]:
renderapi.stack.set_stack_state(f"{stacks['out']}_scaled",
                                state='COMPLETE',
                                **params_render)

<Response [201]>